In [1]:
import json
import pandas as pd
import glob
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
transcripts = []
files = glob.glob('json_output/*.json')
for file in files:
    with open(file) as f:
        transcripts.append([file.split('/')[-1], json.load(f)])

In [1]:
#print(transcripts[0])

In [5]:
transcripts[0][0]

'opioids_tqph0257_opioids_tqph0257.json'

In [6]:
#transcript

In [7]:
transcript_data = []

for t in transcripts:
    if 'results' in t[1].keys():
        for r in t[1]['results']:
            if 'transcript' in r['alternatives'][0].keys():
                #print(r['alternatives'][0]['transcript'], r['alternatives'][0]['confidence'])
                transcript_data.append([t[0].split('.')[0], r['alternatives'][0]['transcript'], r['alternatives'][0]['confidence']])

In [8]:
df = pd.DataFrame(transcript_data, columns=['file_name', 'text', 'confidence'])

In [9]:
df

,file_name,text,confidence
0,opioids_tqph0257_opioids_tqph0257,news reports are coming in from across the cou...,0.826548
1,opioids_tqph0257_opioids_tqph0257,I've some of these other drugs in the pipeline...,0.885013
2,opioids_tqph0257_opioids_tqph0257,product opportunity is the the product for ca...,0.870853
3,opioids_tqph0257_opioids_tqph0257,it was radio 1200 WOAI San Antonio's first ne...,0.772321
4,opioids_tqph0257_opioids_tqph0257,it's called fan Torah cephalon say it does pl...,0.779900
...,...,...,...
177,lnq23e00_VTS_01_1_512kb,first extensive research and dialogue was und...,0.912838
178,lnq23e00_VTS_01_1_512kb,next info tab is offering further assistance ...,0.912838
179,lnq23e00_VTS_01_1_512kb,ultimately your own expertise and knowledge o...,0.912839
180,lnq23e00_VTS_01_1_512kb,seizing the initiative urges you to check on ...,0.912839


In [10]:
df_text = pysqldf("""
SELECT file_name, GROUP_CONCAT(text) as transcript, avg(confidence), min(confidence), max(confidence) 
FROM df
GROUP BY file_name
""")

In [11]:
df_text

,file_name,transcript,avg(confidence),min(confidence),max(confidence)
0,lnq23e00_VTS_01_1_512kb,environmental tobacco smoke ETS is a political...,0.887068,0.715651,0.912839
1,opioids_tqph0257_opioids_tqph0257,news reports are coming in from across the cou...,0.847476,0.772321,0.912839
2,tobacco_sdo23e00_10001665-0-2058500033_chunk_1...,"Yes dear, let's say Philip Morris oh is there ...",0.810598,0.651882,0.912839
3,tobacco_two47c00_VTS_01_1_512kb,environmental tobacco smoke ETS is a political...,0.887068,0.715651,0.912839
4,tobacco_wvp23e00_VTS_01_1_512kb,environmental tobacco smoke ETS is a political...,0.887068,0.715651,0.912839


In [12]:
df_text.to_csv('data/transcript_accuracy.csv', index=False)